# Customer Segmentation Using RFM Model

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing

import time, warnings
import datetime as dt

#visualizations
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns

warnings.filterwarnings("ignore")

## Get the Data

In [ ]:
#load the dataset
retail_df = pd.read_csv("C:/Users/santh/Desktop/Project/Sales_Data1.csv")
retail_df.tail()

## Prepare the Data

In [ ]:
retail_uk = retail_df[retail_df['Country']=='United Kingdom']
#check the shape
retail_uk.shape

In [ ]:
#remove canceled orders
retail_uk = retail_uk[retail_uk['Quantity']>0]
retail_uk.shape

In [ ]:
#remove rows where customerID are NA
retail_uk.dropna(subset=['Customer ID'],how='all',inplace=True)
retail_uk.shape

In [ ]:
#restrict the data to one full year because it's better to use a metric per Months or Years in RFM
retail_uk = retail_uk[retail_uk['InvoiceDate']>= "2010-01-01"]
retail_uk.shape

In [ ]:
print("Summary..")
#exploring the unique values of each attribute
print("Number of transactions: ", retail_uk['Invoice'].nunique())
print("Number of products bought: ",retail_uk['StockCode'].nunique())
print("Number of customers:", retail_uk['Customer ID'].nunique() )
print("Percentage of customers NA: ", round(retail_uk['Customer ID'].isnull().sum() * 100 / len(retail_df),2),"%" )

# RFM Analysis

## Recency

In [ ]:
#last date available in our dataset
retail_uk['InvoiceDate'].max()

In [ ]:
now = dt.date(2011,12,10)
print(now)

In [ ]:
#create a new column called date which contains the date of invoice only
retail_uk['date'] = pd.DatetimeIndex(retail_uk['InvoiceDate']).date
retail_uk.head()

In [ ]:
#group by customers and check last date of purshace
recency_df = retail_uk.groupby(by='Customer ID', as_index=False)['date'].max()
recency_df.columns = ['Customer ID','LastPurshaceDate']
recency_df.head()

In [ ]:
#calculate recency
recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x: (now - x).days)
recency_df.head()

In [ ]:
#drop LastPurchaseDate as we don't need it anymore
recency_df.drop('LastPurshaceDate',axis=1,inplace=True)

## Frequency

In [ ]:
#drop duplicates
retail_uk_copy = retail_uk
retail_uk_copy.drop_duplicates(subset=['Invoice', 'Customer ID'], keep="first", inplace=True)
#calculate frequency of purchases
frequency_df = retail_uk_copy.groupby(by=['Customer ID'], as_index=False)['Invoice'].count()
frequency_df.columns = ['Customer ID','Frequency']
frequency_df.head()

## Monetary

In [ ]:
#create column total cost
retail_uk['TotalCost'] = retail_uk['Quantity'] * retail_uk['Price']

In [ ]:
monetary_df = retail_uk.groupby(by='Customer ID',as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['Customer ID','Monetary']
monetary_df.head()

## Create RFM Table

In [ ]:
#merge recency dataframe with frequency dataframe
temp_df = recency_df.merge(frequency_df,on='Customer ID')
temp_df.head()

In [ ]:
#merge with monetary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df,on='Customer ID')
#use CustomerID as index
rfm_df.set_index('Customer ID',inplace=True)
#check the head
rfm_df.head()

## RFM Table Correctness verification

In [ ]:
retail_uk[retail_uk['Customer ID']=='12820']

In [ ]:
(now - dt.date(2011,9,26)).days == 75

## Customer segments with RFM Model

### RFM Quartiles

In [ ]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

In [ ]:
quantiles.to_dict()

In [ ]:
### Creation of RFM Segments

In [ ]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [ ]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [ ]:
rfm_segmentation.head()

In [ ]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

In [ ]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('Monetary', ascending=False).head(10)

### How many customers do we have in each segment?

In [ ]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))